In [44]:
import pandas as pd
import numpy as np

In [45]:
# Define files
input_file = 'homework.csv'
output_file = 'formatted.csv'
example_file = 'example.csv'

In [46]:
# Options
pd.set_option('display.max_columns', None)

# Extract

In [59]:
# Define dataframes
input_df = pd.read_csv(input_file)
example_df = pd.read_csv(example_file)

/Users/dominicsciarrino/Library/Python/3.8/lib/python/site-packages/IPython/core/interactiveshell.py:3398: DtypeWarning: Columns (6,7,23,43,45,86,87,91,92,122,123,124) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [63]:
# Show input file
input_df.head(1)

,item number,upc,new item,system creation date,min order qty,sales uom,wholesale ($),map ($),msrp ($),description,long description,brand,item category,item type,outdoor,item width (inches),item depth (inches),item height (inches),item diameter (inches),item weight (pounds),multi-piece dimension 1 (inches),multi-piece dimension 2 (inches),multi-piece dimension 3 (inches),multi-piece dimension 4 (inches),item materials,primary color family,item finish,item finish 1,item finish 2,item finish 3,primary image filename,url primary image,url alternate image 1,url alternate image 2,url alternate image 3,url alternate image 4,url alternate image 5,url alternate image 6,url alternate image 7,url alternate image 8,url room setting image 1,url room setting image 2,url room setting image 3,url drawing,url interactive 360 image,url animated gif,url product sheet,url instruction sheet,url marketing sheet 1,url california label (jpg),url california label (pdf),item style,item substyle,item substyle 2,item collection,licensed by,carton count,truck only,carton 1 width (inches),carton 1 length (inches),carton 1 height (inches),carton 1 weight (pounds),carton 1 volume (cubic feet),carton 2 width (inches),carton 2 length (inches),carton 2 height (inches),carton 2 weight (pounds),carton2volumecubicfeet,carton 3 width (inches),carton 3 length (inches),carton 3 height (inches),carton 3 weight (pounds),carton 3 volume (cubic feet),ada compliant,available with eef,conversion kit option,title 24 compliant,safety rating,certified damp/wet,bulb 1 count,bulb 1 wattage,bulb 1 type,bulb 1 base,bulb 1 included,bulb 2 count,bulb 2 wattage,bulb 2 type,bulb 2 base,bulb 2 included,led,total lumens,color temperature,cri,voltage,switch type,dimmable,lamp base dimensions (inches),backplate/canopy dimensions (inches),extension rods (inches),min overall height (inches),max overall height (inches),min extension (inches),max extension (inches),hcwo (inches),shade/glass description,shade/glass materials,shade/glass finish,shade/glass width,shade/glass width at top (inches),shade/glass width at bottom (inches),shade/glass height (inches),shade shape,harp/spider,cord color,cord length (inches),chain length (inches),chain price ($),replacement glass price ($),replacement crystal price ($),mirror width (inches),mirror height (inches),drawer count,drawer 1 interior dimensions (inches),drawer 2 interior dimensions (inches),drawer 3 interior dimensions (inches),furniture arm height (inches),furniture seat height (inches),furniture seat dimensions (inches),furniture weight capacity (pounds),country of origin,primary catalog,primary catalog page,related items,brand bio,helpful tips,selling point 1,selling point 2,selling point 3,selling point 4,selling point 5,selling point 6,selling point 7,selling point 8,selling point 9,selling point 10,record status
0,203,8.109370e+11,NaN,7/7/15,1,Each,228.0,304.0,$456.00,White Obelisk Table Lamp With Night Light,NaN,Dimond Lighting,Indoor Lighting,Table Lamp,No,18.0,18.0,36.0,18.0,6.0,NaN,NaN,NaN,NaN,Glass,NaN,White,White,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://s3-us-west-2.amazonaws.com/elk-assets/...,NaN,NaN,https://s3-us-west-2.amazonaws.com/elk-assets/...,https://s3-us-west-2.amazonaws.com/elk-assets/...,Modern / Contemporary,Luxe / Glam,NaN,Obelisk,NaN,1.0,No,19.0,19.0,34.0,10.0,7.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,No,NaN,UL,NaN,1.0,100.0,A21 3-Way,E26 Medium,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3-Way,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Round Hardback White Fabric Shade,Fabric,White,NaN,16.0,18.0,11.0,NaN,NaN,NaN,66.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,China,ELK Home 2019,529,NaN,Dimond Lighting is an award-winning designer a...,When choosing a table lamp it is important to ...,8-Inch wide base,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Active


## Setting up the output dataframe

In [71]:
# Headers for output df
output_file_headers = list(input_df.columns)
output_df = pd.DataFrame(columns=output_file_headers)

## Cleaning the Input File

In [72]:
# Number of rows in original
input_df.shape[0]

6670

In [73]:
# Any items without a UPC code will not be carried over
input_df = input_df.dropna(axis=0, subset=['upc'])

In [74]:
# Number of rows after cleaning
input_df.shape[0]

6643

# Transform

In [90]:
def direct_copy(input_df, output_df, col_name):
    output_df[col_name] = input_df[col_name]

def convert_to_str_copy(input_df, output_df, col_name):
    output_df[col_name] = input_df[col_name].apply(lambda code: str(int(code)))
    
def convert_date_iso6801_copy(input_df, output_df, col_name):
    output_df[col_name] = pd.to_datetime(input_df[col_name]).dt.strftime('%Y-%m-%d')

# Dates should use ISO 8601
dates_col_list = ['system creation date']

# UPC / Gtin / EAN should be handled as strings
treat_as_string_col_list = ['upc']

# Currency should be rounded to unit of accounting. Assume USD for currency and round to cents.
currency_col_list = [
    'wholesale ($)',
    'map ($)',
    'msrp ($)',
    'chain price ($)',
    'replacement glass price ($)',
    'replacement crystal price ($)'
]
# Dimensions that are in inches
# Preserve as much precision as possible
dimensions_inches_col_list = [
    'item width (inches)',
    'item depth (inches)',
    'item height (inches)',
    'item diameter (inches)',
    'multi-piece dimension 1 (inches)',
    'multi-piece dimension 2 (inches)',
    'multi-piece dimension 3 (inches)',
    'multi-piece dimension 4 (inches)',
    'carton 1 width (inches)',
    'carton 1 length (inches)',
    'carton 1 height (inches)',
    'carton 2 width (inches)',
    'carton 2 length (inches)',
    'carton 2 height (inches)',
    'carton 3 width (inches)',
    'carton 3 length (inches)',
    'carton 3 height (inches)',
    'lamp base dimensions (inches)',
    'backplate/canopy dimensions (inches)',
    'extension rods (inches)',
    'min overall height (inches)',
    'max overall height (inches)',
    'min extension (inches)',
    'max extension (inches)',
    'hcwo (inches)',
    'shade/glass width at top (inches)',
    'shade/glass width at bottom (inches)',
    'shade/glass height (inches)',
    'cord length (inches)',
    'chain length (inches)',
    'mirror width (inches)',
    'mirror height (inches)',
    'drawer 1 interior dimensions (inches)',
    'drawer 2 interior dimensions (inches)',
    'drawer 3 interior dimensions (inches)',
    'furniture arm height (inches)',
    'furniture seat height (inches)',
    'furniture seat dimensions (inches)',
    'shade/glass width' # Dimensions without inches, assume inches
]
# Weight that is in pounds
# Preserve as much precision as possible
weight_pounds_col_list = [
    'item weight (pounds)',
    'carton 1 weight (pounds)',
    'carton 2 weight (pounds)',
    'carton 3 weight (pounds)',
    'furniture weight capacity (pounds)'
]
# Convert anything that isn't inches to inches
# Cubic feet must be converted to cubic inches
convert_cubic_feet_to_cubic_inches = [
    'carton 1 volume (cubic feet)',
    'carton 2 volume (cubic feet)',
    'carton 3 volume (cubic feet)'
]


for col in list(input_df.columns):
    if col in dates_col_list:
        convert_date_iso6801_copy(input_df, output_df, col)
    elif col in treat_as_string_col_list:
        convert_to_str_copy(input_df, output_df, col)
    else:
        direct_copy(input_df, output_df, col)



In [92]:
output_df

,item number,upc,new item,system creation date,min order qty,sales uom,wholesale ($),map ($),msrp ($),description,long description,brand,item category,item type,outdoor,item width (inches),item depth (inches),item height (inches),item diameter (inches),item weight (pounds),multi-piece dimension 1 (inches),multi-piece dimension 2 (inches),multi-piece dimension 3 (inches),multi-piece dimension 4 (inches),item materials,primary color family,item finish,item finish 1,item finish 2,item finish 3,primary image filename,url primary image,url alternate image 1,url alternate image 2,url alternate image 3,url alternate image 4,url alternate image 5,url alternate image 6,url alternate image 7,url alternate image 8,url room setting image 1,url room setting image 2,url room setting image 3,url drawing,url interactive 360 image,url animated gif,url product sheet,url instruction sheet,url marketing sheet 1,url california label (jpg),url california label (pdf),item style,item substyle,item substyle 2,item collection,licensed by,carton count,truck only,carton 1 width (inches),carton 1 length (inches),carton 1 height (inches),carton 1 weight (pounds),carton 1 volume (cubic feet),carton 2 width (inches),carton 2 length (inches),carton 2 height (inches),carton 2 weight (pounds),carton2volumecubicfeet,carton 3 width (inches),carton 3 length (inches),carton 3 height (inches),carton 3 weight (pounds),carton 3 volume (cubic feet),ada compliant,available with eef,conversion kit option,title 24 compliant,safety rating,certified damp/wet,bulb 1 count,bulb 1 wattage,bulb 1 type,bulb 1 base,bulb 1 included,bulb 2 count,bulb 2 wattage,bulb 2 type,bulb 2 base,bulb 2 included,led,total lumens,color temperature,cri,voltage,switch type,dimmable,lamp base dimensions (inches),backplate/canopy dimensions (inches),extension rods (inches),min overall height (inches),max overall height (inches),min extension (inches),max extension (inches),hcwo (inches),shade/glass description,shade/glass materials,shade/glass finish,shade/glass width,shade/glass width at top (inches),shade/glass width at bottom (inches),shade/glass height (inches),shade shape,harp/spider,cord color,cord length (inches),chain length (inches),chain price ($),replacement glass price ($),replacement crystal price ($),mirror width (inches),mirror height (inches),drawer count,drawer 1 interior dimensions (inches),drawer 2 interior dimensions (inches),drawer 3 interior dimensions (inches),furniture arm height (inches),furniture seat height (inches),furniture seat dimensions (inches),furniture weight capacity (pounds),country of origin,primary catalog,primary catalog page,related items,brand bio,helpful tips,selling point 1,selling point 2,selling point 3,selling point 4,selling point 5,selling point 6,selling point 7,selling point 8,selling point 9,selling point 10,record status
0,203,810937002035,NaN,2015-07-07,1,Each,228.0,304.0,$456.00,White Obelisk Table Lamp With Night Light,NaN,Dimond Lighting,Indoor Lighting,Table Lamp,No,18.0,18.0,36.0,18.0,6.0,NaN,NaN,NaN,NaN,Glass,NaN,White,White,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://s3-us-west-2.amazonaws.com/elk-assets/...,NaN,NaN,https://s3-us-west-2.amazonaws.com/elk-assets/...,https://s3-us-west-2.amazonaws.com/elk-assets/...,Modern / Contemporary,Luxe / Glam,NaN,Obelisk,NaN,1.0,No,19.0,19.0,34.0,10.0,7.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,No,NaN,UL,NaN,1.0,100.0,A21 3-Way,E26 Medium,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3-Way,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Round Hardback White Fabric Shade,Fabric,White,NaN,16.0,18.0,11.0,NaN,NaN,NaN,66.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,China,ELK Home 2019,529,NaN,Dimond Lighting is an award-winning designer a...,When choosing a table lamp it is important to ...,8-Inch wide base,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Active
1,205,810937002059,NaN,2015-07-07,1,Each,229.5,306.0,$459.00,Antiqued Porcelain Table Lamp In Celadon Crack...,NaN,Di